In [2]:
import pandas as pd 
import polars as pl 
import numpy as np
import os
import re
import sys
import time

from polars import col, lit, when

In [2]:
%pip install polars

  Using cached polars-0.17.14-cp37-abi3-macosx_11_0_arm64.whl (15.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [4]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

Could not import azure.core python package.


In [9]:
agent = create_csv_agent(OpenAI(temperature=0), 'data/Metro_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv', verbose=True)

In [10]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 896
Thought: I now know the final answer
Final Answer: 896 rows

> Finished chain.


'896 rows'

In [11]:
agent.run("summarize the contents of the table")



> Entering new AgentExecutor chain...
Thought: I need to look at the data and see what it is telling me
Action: python_repl_ast
Action Input: df.describe()
Observation:             RegionID    SizeRank  2023-05-31  2023-07-31  2024-04-30
count     896.000000  896.000000  896.000000  896.000000  896.000000
mean   412080.852679  461.632812    0.386049    0.739063    4.354464
std     78335.582250  268.584051    0.464199    0.927743    2.575428
min    102001.000000    0.000000   -1.600000   -3.300000   -5.600000
25%    394546.500000  230.750000    0.100000    0.300000    2.800000
50%    394795.500000  459.500000    0.400000    0.800000    4.500000
75%    395045.250000  689.250000    0.600000    1.300000    6.000000
max    753929.000000  939.000000    2.100000    5.100000   15.900000
Thought: I can see that the data is about regions in the US, and the columns are related to population growth in different states
Final Answer: The dataframe contains information about regions in the US, with

'The dataframe contains information about regions in the US, with columns related to population growth in different states.'

In [12]:
agent.run("What are the five fastest growing regions?")



> Entering new AgentExecutor chain...
Thought: I need to find the regions with the highest growth rate.
Action: python_repl_ast
Action Input: df.sort_values(by='2024-04-30', ascending=False).head()
Observation:      RegionID  SizeRank         RegionName RegionType StateName    BaseDate   
715    394894       734         Murray, KY        msa        KY  2023-04-30  \
886    394342       929       Atchison, KS        msa        KS  2023-04-30   
873    394965       911        Parsons, KS        msa        KS  2023-04-30   
859    394837       895      Maryville, MO        msa        MO  2023-04-30   
802    394892       832  Mountain Home, ID        msa        ID  2023-04-30   

     2023-05-31  2023-07-31  2024-04-30  
715         1.4         5.1        15.9  
886         2.0         3.4        15.3  
873         1.9         3.3        13.3  
859         2.1         4.4        12.9  
802         1.1         2.6        12.1  
Thought: I now know the five fastest growing regions.
Final 

'The five fastest growing regions are Murray, KY; Atchison, KS; Parsons, KS; Maryville, MO; and Mountain Home, ID.'

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
llm = OpenAI(temperature=0)